# DRMS Basics

DRMS data are organized in data series containing multiple records. The metadata for each individual record is stored in a SQL database, while the actual data are stored as separate files (e.g. FITS files) and usually do not contain any metadata.

More information on DRMS series and JSOC data access is available on the JSOC website:

- http://jsoc.stanford.edu/jsocwiki/JsocDataAccess
- http://jsoc.stanford.edu/ajax/RecordSetHelp.html
- http://jsoc.stanford.edu/jsocwiki/DataSeries

An interactive webinterface for accessing the DRMS can be found at:

- http://jsoc.stanford.edu/ajax/lookdata.html

In [ ]:
import drms
import matplotlib.pyplot as plt
from matplotlib import dates
%matplotlib notebook

Create a [`drms.Client`](http://drms.readthedocs.io/en/stable/generated/drms.Client.html) instance to access the JSOC DRMS server:

In [ ]:
client = drms.Client()
print(client)

## List DRMS series

A list of available series can be obtained using the [`drms.Client.series()`](http://drms.readthedocs.io/en/stable/generated/drms.Client.series.html) method. By default this method returns all available series (more than 200 publicly available from JSOC). A subset can be selected by specifying a regular expression.

Note that series names are case insensitive and that you might need to escape characters like `'.'`.

In [ ]:
client.series('45s')

In [ ]:
client.series('aia\.')

In [ ]:
client.series('hmi\.v_')

## Get detailed series information

Information about a series can be retrieved using [`drms.Client.info()`](http://drms.readthedocs.io/en/stable/generated/drms.Client.info.html).

In [ ]:
info_dopp = client.info('hmi.v_45s')
print(info_dopp)

In [ ]:
info_dopp.primekeys

In [ ]:
info_dopp.keywords.index

In [ ]:
info_dopp.keywords.loc[['T_REC', 'CRLT_OBS', 'OBS_VR']]

In [ ]:
info_dopp.segments

In [ ]:
info_sharp = client.info('hmi.sharp_720s')
print(info_sharp)

In [ ]:
info_sharp.primekeys

In [ ]:
info_sharp.segments

There also exist shortcuts to obtain lists of keywords ([`drms.Client.keys()`](http://drms.readthedocs.io/en/stable/generated/drms.Client.keys.html)) and primekeys ([`drms.Client.pkeys()`](http://drms.readthedocs.io/en/stable/generated/drms.Client.pkeys.html)):

In [ ]:
print(client.pkeys('hmi.v_45s'))

In [ ]:
print(client.keys('hmi.v_45s'))

## Querying metadata

The central command for querying the DRMS is given by the [`drms.Client.query()`](http://drms.readthedocs.io/en/stable/generated/drms.Client.query.html) function. This function can be used to obtain metadata, as well as the location of locally stored data segments. Here we are look at the metadata.

A DRMS recordset can be specified using the series name, followed by one or more pairs of square brackets, which correspond to the primekeys of the specific series. Each bracket group can be used to select a range of records using a primekey expression. A few examples for primekeys that represent a timestamp are:

- `"[2016.01.01_12:00_TAI]"` -> One record at Noon Jan 01, 2016
- `"[2016.01.01_12:00_TAI/1h]"` -> All records for one hour starting at Noon on Jan 01, 2016
- `"[2016.01.01_TAI/7d@1d]"` -> One record per day, for a whole week, starting at Midnight on Jan 01, 2016
- `"[2016.01.01_TAI-2017.01.01_TAI@12h]"` -> One record every 12 hours, for the whole year of 2016

More details and more examples can be found on the [JSOC RecordSet Help](http://jsoc.stanford.edu/ajax/RecordSetHelp.html) page.

In [ ]:
keys = client.query('hmi.m_720s[2017.01.01_TAI/1d@6h]', key=['T_REC', 'OBS_VR', 'DSUN_OBS'])
print(keys)

In [ ]:
keys = client.query('hmi.m_720s[2017.01.01_TAI/1d]', key=['T_REC', 'OBS_VR', 'DSUN_OBS'])
t_rec = drms.to_datetime(keys['T_REC'])

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(8, 6))

ax[0].plot(t_rec, keys['DSUN_OBS'])
ax[0].set_ylabel('Distance [m]')

ax[1].plot(t_rec, keys['OBS_VR'])
ax[1].set_ylabel('Radial velocity [m/s]')

ax[1].set_xlabel('Time')
ax[1].xaxis.set_major_formatter(dates.DateFormatter('%H:%M'))
fig.tight_layout()
fig.show()

In [ ]:
keys = client.query('hmi.m_720s[2015.01.01_TAI/365d@3h]', key=['T_REC', 'DSUN_OBS'])
t_rec = drms.to_datetime(keys['T_REC'])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(t_rec, keys['DSUN_OBS'])
ax.set_xlabel('Time')
ax.set_ylabel('Distance [m]')
fig.tight_layout()
fig.show()

## More interesting metadata

This is a simplified version of [`plot_polarfield.py`](https://github.com/kbg/drms/blob/master/examples/plot_polarfield.py) which is included in the `drms` package.

In [ ]:
series = 'hmi.meanpf_720s'
tsel = '2010.05.01_TAI-2017.01.01_TAI@12h'

qstr = '{}[{}]'.format(series, tsel)
print('Querying "{}"...'.format(qstr))

keys = client.query(qstr, key=['T_REC', 'CAPN2', 'CAPS2'])
print('-> {} lines retrieved.'.format(len(keys)))

t_rec = drms.to_datetime(keys['T_REC'])

In [ ]:
info_meanpf = client.info(series)
info_meanpf.keywords.loc[['CAPN2', 'CAPS2']].note

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(t_rec, keys['CAPN2'], label='North')
ax.plot(t_rec, keys['CAPS2'], label='South')
ax.set_ylim(-6.2, 6.2)
ax.set_xlabel('Year')
ax.set_ylabel('Mean radial field strength [G]')
ax.legend()
fig.tight_layout()
fig.show()